In [1]:
import os
import glob
import shutil

import lzma
import json

import numpy as np
import pandas as pd

import geopandas as gpd
from geopandas.tools import overlay
import geopy

# Data

In [2]:
# directory = '#ruedeparis'
directory = '#paris'
# directory = '#eolienne'

In [3]:
row_odeur = []
with open('odeur.txt', 'r') as file :
    for line in file :
        if len(str(line[:-1])) != 0:
            row_odeur.append(str(line[:-1]))
            
odeurs = []
for mot in row_odeur:
#     print(mot)
    mot = mot.replace(" ", "")
    mot = mot.replace("Ã©", "é")
    mot = mot.replace("Ã»", "u")
    mot = mot.replace("Ã¨", "è")
    mot = mot.replace("Ãª", "ê")
    mot = mot.replace("Ã\xa0", "à")
    mot = mot.replace("Ã‰", "E")
    mot = mot.replace("Ã®", "i")
    mot = mot.replace("/", "")
    mot = mot.replace("'", "")
    odeurs.append(mot)

odeurs.remove("Abbatoir")
odeurs.remove("Produitsanesthésiants")
odeurs.remove("Unarrogantchauve")
odeurs.remove("Pinartificiel")
odeurs.remove("Caoutchoucchauffé")
odeurs.remove("Pipeàbulles")
odeurs.remove("Combustiblebrulé")
odeurs.remove("Allumettebrulée")
odeurs.remove("Lecaoutchoucbrulé")
odeurs.remove("Peinturechimique")
odeurs.remove("Futdecigarette")
odeurs.remove("Liquidedenettoyage")
odeurs.remove("Urinesèche")
odeurs.remove("Caveàvinvide")
odeurs.remove("Acélatedéthyle")
odeurs.remove("Conservesdefigues")
odeurs.remove("Fleurcoupéefraiche")
odeurs.remove("Déchetsfraisaveclégumes")
odeurs.remove("Unitéindustrielle")
odeurs.remove("SitedeLeesy")

In [4]:
list_list_xz = []
for i in range(len(odeurs)):
    directory = '#' + str(odeurs[i])
    list_list_xz.append(glob.glob('./'+ directory + '/*.xz'))
    print(len(list_list_xz[i]))
    print(list_list_xz[i][0])

1
./#Acideacétique\2020-03-24_12-14-26_UTC.json.xz
200
./#Acide\2020-06-03_21-06-56_UTC.json.xz
200
./#Désodorisant\2019-09-19_10-12-48_UTC.json.xz
300
./#Alcool\2020-02-24_11-53-47_UTC.json.xz
205
./#Alcoolique\2016-05-01_15-54-08_UTC.json.xz
300
./#Amande\2020-03-01_10-34-59_UTC.json.xz
300
./#Ambre\2019-02-25_10-04-44_UTC.json.xz
183
./#Animal\2020-02-11_18-09-57_UTC.json.xz
301
./#anis\2019-12-23_11-54-35_UTC.json.xz
300
./#Pomme\2018-05-29_15-41-09_UTC.json.xz
301
./#Abricot\2020-02-20_08-42-19_UTC.json.xz
300
./#Eau\2016-06-30_00-29-41_UTC.json.xz
201
./#Aisselle\2020-05-06_19-46-25_UTC.json.xz
201
./#Aromatique\2020-06-12_10-14-51_UTC.json.xz
299
./#Artichaut\2020-06-09_07-40-54_UTC.json.xz
6
./#Fruitsartificiels\2019-06-15_13-10-47_UTC.json.xz
201
./#Cendres\2017-05-14_19-46-34_UTC.json.xz
300
./#Asperges\2020-06-06_07-26-04_UTC.json.xz
201
./#Asphalte\2018-10-07_09-43-11_UTC.json.xz
201
./#Fumées\2019-04-13_18-12-36_UTC.json.xz
300
./#Automne\2016-11-12_16-43-47_UTC.json.xz
29

IndexError: list index out of range

# Localisation

## Adresse

In [5]:
directory = '#parisstreet'
list_xz = glob.glob('./'+ directory + '/*.xz')
# list_xz = list_list_xz[5]
len(list_xz)

8762

In [6]:
adress = dict()
for file in list_xz:
    file_json = json.load(lzma.open(file))
    location = file_json['node']['location']
    if location :
        if location['address_json']:
            adress[file] = json.loads(location['address_json'])
        
df_adress = pd.DataFrame.from_dict(adress, orient = 'index')
df_adress['espace'] = ' '

In [7]:
print(df_adress.size)
df_adress

54747


,street_address,zip_code,city_name,region_name,country_code,exact_city_match,exact_region_match,exact_country_match,espace
./#parisstreet\2016-01-18_00-34-50_UTC.json.xz,,,"Paris, France",,FR,True,False,False,
./#parisstreet\2016-05-09_05-07-58_UTC.json.xz,Pont d'Arcole,75004,"Paris, France",,FR,False,False,False,
./#parisstreet\2016-10-08_22-03-56_UTC.json.xz,Denfert rochereau,75014,,,,False,False,False,
./#parisstreet\2017-03-26_22-27-14_UTC.json.xz,,,"Paris, France",,FR,True,False,False,
./#parisstreet\2017-06-08_17-06-54_UTC.json.xz,,,"Paris, France",,FR,True,False,False,
...,...,...,...,...,...,...,...,...,...
./#parisstreet\2020-07-31_16-29-18_UTC.json.xz,,75003,"Paris, France",,FR,False,False,False,
./#parisstreet\2020-07-31_16-30-49_UTC.json.xz,,,"Paris, France",,FR,True,False,False,
./#parisstreet\2020-07-31_17-45-56_UTC.json.xz,,,"Paris, France",,FR,True,False,False,
./#parisstreet\2020-07-31_18-34-31_UTC.json.xz,,,"Paris, France",,FR,False,False,False,


In [8]:
#Par ville
# df_adress['adress'] = df_adress['city_name']

#Par zip
# df_adress['adress'] = df_adress['zip_code'] 

# Par adresse
df_adress['adress'] = df_adress['street_address'] +  df_adress['espace']  + df_adress['city_name']

In [9]:
df_adress['adress'] = df_adress['adress'].apply(lambda x : x if len(x)>0 else np.nan)
df_adress['street_address'] = df_adress['street_address'].apply(lambda x : x if len(x)>0 else np.nan)


df_adress = df_adress.dropna(subset=['street_address'])
df_adress.describe()

,street_address,zip_code,city_name,region_name,country_code,exact_city_match,exact_region_match,exact_country_match,espace,adress
count,1746,1746,1746,1746,1746,1746,1746,1746,1746,1746
unique,516,93,55,1,23,1,1,1,1,519
top,Cité Du Sacré Coeur,75004,"Paris, France",,FR,False,False,False,,"Cité Du Sacré Coeur Paris, France"
freq,119,261,1634,1746,1672,1746,1746,1746,1746,119


In [10]:
df_adress.head()

,street_address,zip_code,city_name,region_name,country_code,exact_city_match,exact_region_match,exact_country_match,espace,adress
./#parisstreet\2016-05-09_05-07-58_UTC.json.xz,Pont d'Arcole,75004,"Paris, France",,FR,False,False,False,,"Pont d'Arcole Paris, France"
./#parisstreet\2016-10-08_22-03-56_UTC.json.xz,Denfert rochereau,75014,,,,False,False,False,,Denfert rochereau
./#parisstreet\2018-03-15_22-49-50_UTC.json.xz,9 Place de la République,75003,"Paris, France",,FR,False,False,False,,"9 Place de la République Paris, France"
./#parisstreet\2018-07-01_13-59-49_UTC.json.xz,Pont de la Concorde,75007,"Paris, France",,FR,False,False,False,,"Pont de la Concorde Paris, France"
./#parisstreet\2018-07-21_19-31-22_UTC.json.xz,Musée du Louvre,75001,"Paris, France",,FR,False,False,False,,"Musée du Louvre Paris, France"


## Latitude & Longitude

In [11]:
from geopy.extra.rate_limiter import RateLimiter

from geopy.geocoders import Nominatim
from geopy.geocoders import BANFrance


df = df_adress.copy()

# locator = Nominatim(user_agent="myGeocoder") #OpenStreetMap
locator = BANFrance(user_agent="myGeocoder", domain='api-adresse.data.gouv.fr') #Base Adresse Nationale France API.

# 1 - fonction pour obtenir la localisation (1s par ligne)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2-  créer la colonne location
df['location'] = df['adress'].apply(geocode)
# 3 - créer longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else np.nan)
# 4 - efface les lignes sans adresses trouvées
df = df.dropna()
# 5 - créer une colonne lat & long
df[['latitude','longitude', 'laltitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

df = df.drop(['street_address','city_name','region_name','country_code','exact_city_match','exact_region_match','exact_country_match','adress','location','point'], axis = 1)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Cité Du Sacré Coeur Paris, France',), **{}).
Traceback (most recent call last):
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 1321, in do_open
    r = h.getresponse()
  File "C:\Users\Basile\anaconda3\envs\geo_env\li

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Rue Spontini Paris, France',), **{}).
Traceback (most recent call last):
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 1321, in do_open
    r = h.getresponse()
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\http\

RateLimiter caught an error, retrying (0/2 tries). Called with (*('11  Boulevard Montmartre Paris, France',), **{}).
Traceback (most recent call last):
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 1321, in do_open
    r = h.getresponse()
  File "C:\Users\Basile\anaconda3\envs\geo_e

RateLimiter caught an error, retrying (0/2 tries). Called with (*("Pont de L' Archevêché Paris, France",), **{}).
Traceback (most recent call last):
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\Basile\anaconda3\envs\geo_env\lib\urllib\request.py", line 1321, in do_open
    r = h.getresponse()
  File "C:\Users\Basile\anaconda3\envs\geo_env\

In [22]:
geo_adress = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326" )
geo_adress.to_csv('post_BAN.csv')
geo_adress = geo_adress.reset_index()
# geo_adress = geo_adress.drop(['zip_code'], axis = 1)
# geo_adress = geo_adress.drop(['file'], axis = 1)
geo_adress['file'] = geo_adress['index'] 
show = geo_adress.drop(['file', 'espace', 'laltitude'], axis = 1)

In [23]:
show.head()

,index,zip_code,latitude,longitude,geometry
0,./#parisstreet\2016-05-09_05-07-58_UTC.json.xz,75004,48.854836,2.350256,POINT (2.35026 48.85484)
1,./#parisstreet\2016-10-08_22-03-56_UTC.json.xz,75014,44.017829,1.360620,POINT (1.36062 44.01783)
2,./#parisstreet\2018-03-15_22-49-50_UTC.json.xz,75003,48.866667,2.364003,POINT (2.36400 48.86667)
3,./#parisstreet\2018-07-01_13-59-49_UTC.json.xz,75007,48.866727,2.322939,POINT (2.32294 48.86673)
4,./#parisstreet\2018-07-21_19-31-22_UTC.json.xz,75001,48.859301,2.340771,POINT (2.34077 48.85930)


In [14]:
geo_adress.describe()

,latitude,longitude,laltitude
count,1694.000000,1694.000000,1694.0
mean,48.665339,2.373146,0.0
std,2.592553,2.210689,0.0
min,-21.029956,-52.653542,0.0
25%,48.855534,2.322925,0.0
50%,48.860307,2.341159,0.0
75%,48.872916,2.351132,0.0
max,49.647836,55.513332,0.0


In [19]:
1583/8762

0.18066651449440768

In [17]:
#MASK

#France
# geospatial_data  = 'C:\\Users\\Basile\\Pictures\\DATA\\france-geojson-master\\metropole-version-simplifiee.geojson'

# Paris
geospatial_data  = 'C:\\Users\\Basile\\Pictures\\DATA\\france-geojson-master\\departements\\75-paris\\departement-75-paris.geojson'

geo = gpd.read_file(geospatial_data, crs="EPSG:4326" )
geo_adress_mask = overlay(geo_adress, geo)
geo_adress_mask = geo_adress_mask.drop(['index', 'espace', 'nom', 'code'], axis = 1)

In [24]:
geo_adress_mask.describe()

,latitude,longitude,laltitude
count,1583.000000,1583.000000,1583.0
mean,48.864417,2.336145,0.0
std,0.013293,0.025002,0.0
min,48.820109,2.228874,0.0
25%,48.855681,2.325696,0.0
50%,48.860307,2.341159,0.0
75%,48.872666,2.350434,0.0
max,48.898176,2.398201,0.0


## Save the preprocessing

In [25]:
list_preprocess = geo_adress_mask['file']

### Si adresse deja enregistrées

In [26]:
# #Si adresse deja enregistrées
# import os
# import pandas as pd

# path_here = os.getcwd()
# output_dir = os.path.join(path_here,directory[1:])
# list_preprocess = pd.read_csv(os.path.join(output_dir,'df_adress.csv'))

### Fin du si 

In [27]:
def savePreProcessing(output_path, jpg = False):
    i = 0
    error = 0
    for file in list_xz:
        if file in list_preprocess.values:
            i = i + 1
            path = os.path.split(file)[0]
            
            print()
            
            #json
            file_json = os.path.split(file)[-1]
            output_json = os.path.join(output_path, file_json)
            shutil.copy(file,output_json)
            
            #jpg
            if jpg :
                file_jpg    = file_json[:-8] + '.jpg'
                in_file_jpg = os.path.join(path,file_jpg)
                output_jpg  = os.path.join(output_path, file_jpg)
                if os.path.isfile(in_file_jpg):
                    shutil.copy(in_file_jpg,output_jpg)
                else :
                    file_jpg = file_json[:-8] + '_1' + '.jpg'
                    in_file_jpg = os.path.join(path,file_jpg)
                    if os.path.isfile(in_file_jpg):
                        shutil.copy(in_file_jpg,output_jpg)
                    else :
                        error = error + 1
    print('nombre erreurs : ' + str(error))
            
    print('{} post ont été enregistrés'.format(i))

In [29]:
df = pd.DataFrame({'zip_code': [75004], 'latitude': [48.854836], 'longitude': [2.350256	], 'laltitude': [0], 'file' : ['./#parisstreet\2016-05-09_05-07-58_UTC.json.xz']})
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326" )
gdf

,zip_code,latitude,longitude,laltitude,file,geometry
0,75004,48.854836,2.350256,0,./#parisstreet6-05-09_05-07-58_UTC.json.xz,POINT (2.35026 48.85484)


In [30]:
geo_adress_mask

,zip_code,latitude,longitude,laltitude,file,geometry
0,75004,48.854836,2.350256,0.0,./#parisstreet\2016-05-09_05-07-58_UTC.json.xz,POINT (2.35026 48.85484)
1,75003,48.866667,2.364003,0.0,./#parisstreet\2018-03-15_22-49-50_UTC.json.xz,POINT (2.36400 48.86667)
2,75007,48.866727,2.322939,0.0,./#parisstreet\2018-07-01_13-59-49_UTC.json.xz,POINT (2.32294 48.86673)
3,75001,48.859301,2.340771,0.0,./#parisstreet\2018-07-21_19-31-22_UTC.json.xz,POINT (2.34077 48.85930)
4,75001,48.859301,2.340771,0.0,./#parisstreet\2018-07-21_19-32-13_UTC.json.xz,POINT (2.34077 48.85930)
...,...,...,...,...,...,...
1578,75020,48.859789,2.390837,0.0,./#parisstreet\2020-07-30_21-24-24_UTC.json.xz,POINT (2.39084 48.85979)
1579,,48.869057,2.324187,0.0,./#parisstreet\2020-07-30_21-46-02_UTC.json.xz,POINT (2.32419 48.86906)
1580,75007,48.858819,2.294597,0.0,./#parisstreet\2020-07-30_22-50-54_UTC.json.xz,POINT (2.29460 48.85882)
1581,75013,48.887099,2.342330,0.0,./#parisstreet\2020-07-31_06-29-45_UTC.json.xz,POINT (2.34233 48.88710)


In [31]:
test = pd.concat([geo_adress_mask, gdf])
test

,zip_code,latitude,longitude,laltitude,file,geometry
0,75004,48.854836,2.350256,0.0,./#parisstreet\2016-05-09_05-07-58_UTC.json.xz,POINT (2.35026 48.85484)
1,75003,48.866667,2.364003,0.0,./#parisstreet\2018-03-15_22-49-50_UTC.json.xz,POINT (2.36400 48.86667)
2,75007,48.866727,2.322939,0.0,./#parisstreet\2018-07-01_13-59-49_UTC.json.xz,POINT (2.32294 48.86673)
3,75001,48.859301,2.340771,0.0,./#parisstreet\2018-07-21_19-31-22_UTC.json.xz,POINT (2.34077 48.85930)
4,75001,48.859301,2.340771,0.0,./#parisstreet\2018-07-21_19-32-13_UTC.json.xz,POINT (2.34077 48.85930)
...,...,...,...,...,...,...
1579,,48.869057,2.324187,0.0,./#parisstreet\2020-07-30_21-46-02_UTC.json.xz,POINT (2.32419 48.86906)
1580,75007,48.858819,2.294597,0.0,./#parisstreet\2020-07-30_22-50-54_UTC.json.xz,POINT (2.29460 48.85882)
1581,75013,48.887099,2.342330,0.0,./#parisstreet\2020-07-31_06-29-45_UTC.json.xz,POINT (2.34233 48.88710)
1582,,48.898115,2.336921,0.0,./#parisstreet\2020-07-31_09-14-07_UTC.json.xz,POINT (2.33692 48.89811)


In [32]:
path_here = os.getcwd()
output_dir = os.path.join(path_here,directory[1:])
if not(os.path.exists(output_dir)):
    os.mkdir(output_dir)

output_prepro = os.path.join(output_dir,'preprocessing')
if not(os.path.exists(output_prepro)):
    os.mkdir(output_prepro)

In [33]:
geo_adress_mask.to_csv(os.path.join(output_dir,'df_adress.csv'), index = False)

In [34]:
savePreProcessing(output_prepro, jpg = True)